In [1]:
import requests
import json
import pandas as pd
from fpdf import FPDF

In [2]:
def extract():
    api_key = eval(open('config.txt').read())['auth']
    player_stats = requests.get(
        f'https://api.sportsdata.io/v3/nba/stats/json/PlayerSeasonStatsByTeam/2023/MIA?key={api_key}').json()
    return player_stats


def write_to_json_file(data):
    with open('nba_stats.json', 'w') as f:
        json.dump(data, f)


def transform(data):
    df = pd.DataFrame(data)
    interesting_columns = ['Season', 'Team', 'Position', 'Games', 'FantasyPoints', 'Minutes', 'TwoPointersMade', 'TwoPointersAttempted', 'TwoPointersPercentage', 'ThreePointersMade', 'ThreePointersAttempted', 'ThreePointersPercentage', 'FreeThrowsMade', 'FreeThrowsAttempted', 'FreeThrowsPercentage', 'OffensiveRebounds', 'DefensiveRebounds', 'Rebounds', 'Assists', 'Steals', 'BlockedShots', 'Turnovers', 'PersonalFouls', 'Points', 'PlayerEfficiencyRating']
    # df = df.pivot_table(index=['Name'], values=interesting_columns, sort=False)
    df = df.set_index('Name')
    df = df[interesting_columns]
    column_names = {
        'Season': 'Season',
        'Team': 'Team',
        'Position': 'Pos',
        'Games': 'G',
        'FantasyPoints': 'FP',
        'Minutes': 'Min',
        'TwoPointersMade': '2PM',
        'TwoPointersAttempted': '2PA',
        'TwoPointersPercentage': '2P%',
        'ThreePointersMade': '3PM',
        'ThreePointersAttempted': '3PA',
        'ThreePointersPercentage': '3P%',
        'FreeThrowsMade': 'FTM',
        'FreeThrowsAttempted': 'FTA',
        'FreeThrowsPercentage': 'FT%',
        'OffensiveRebounds': 'OffReb',
        'DefensiveRebounds': 'DefReb',
        'Rebounds': 'Reb',
        'Assists': 'Ast',
        'Steals': 'Stl',
        'BlockedShots': 'Blk',
        'Turnovers': 'TO',
        'PersonalFouls': 'PF',
        'Points': 'Pts',
        'PlayerEfficiencyRating': 'Rtg'
    }
    df.index.name = None
    for _ in range(len(interesting_columns)):
        df.rename(columns=column_names, inplace=True)
    # Change column names
    df.sort_values(by=['FP'], ascending=False, inplace=True)
    # with open('stats_nba.csv', 'w') as f:
    #     df.to_csv(f)
    return df


def create_pdf(df):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(40, 10, 'Los Angeles Lakers')
    pdf.ln()
    pdf.ln()
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(40, 10, 'Season 2022')
    pdf.ln()
    pdf.set_font('Arial', 'B', 10)
    pdf.cell(40, 10, 'Player stats')
    pdf.ln()
    lengths = {
        'Season': 9,
        'Team': 7,
        'Pos': 6,
        'G': 4,
        'FP': 8,
        'Min': 7,
        '2PM': 7,
        '2PA': 7,
        '2P%': 6,
        '3PM': 7,
        '3PA': 7,
        '3P%': 6,
        'FTM': 7,
        'FTA': 7,
        'FT%': 6,
        'OffReb': 9,
        'DefReb': 9,
        'Reb': 7,
        'Ast': 7,
        'Stl': 6,
        'Blk': 6,
        'TO': 7,
        'PF': 7,
        'Pts': 7,
        'Rtg': 6
    }
    pdf.set_font('Arial', 'B', 5)
    pdf.cell(18, 8, 'Player', border=1)
    for key, value in lengths.items():
        pdf.cell(value, 8, key, border=1)
    pdf.ln()
    pdf.set_font('Arial', '', 5)
    for index, row in df.iterrows():
        pdf.cell(18, 4, index, border=1)
        for key, value in lengths.items():
            pdf.cell(value, 4, str(row[key]), border=1)
        pdf.ln()
    pdf.multi_cell(0, 2, '\n* Pos = Position, G = Games, FP = Fantasy Points, Min = Minutes, 2PM = Two Pointers Made, 2PA = Two Pointers Attempted, 2P% = Two Pointers Percentage, 3PM = Three Pointers Made, 3PA = Three Pointers Attempted, 3P% = Three Pointers Percentage, FTM = Free Throws Made, FTA = Free Throws Attempted, FT% = Free Throws Percentage, OffReb = Offensive Rebounds, DefReb = Defensive Rebounds, Reb = Rebounds, Ast = Assists, Stl = Steals, Blk = Blocks, TO = Turnovers, PF = Personal Fouls, Pts = Points, Rtg = Player Efficiency Rating')
    pdf.output('nba_stats.pdf', 'F')
    

In [3]:
stats = extract()


In [4]:
write_to_json_file(stats)


In [5]:
df = transform(stats)
display(df)


,Season,Team,Pos,G,FP,Min,2PM,2PA,2P%,3PM,...,OffReb,DefReb,Reb,Ast,Stl,Blk,TO,PF,Pts,Rtg
Bam Adebayo,2023,MIA,C,17,794.6,764,160.1,307.3,67.8,0.0,...,57.3,149.7,207.0,74.2,26.0,19.5,72.9,76.8,416.6,27.2
Kyle Lowry,2023,MIA,PG,19,757.6,900,45.6,102.9,57.7,62.5,...,16.9,102.9,119.8,147.1,27.3,16.9,66.4,70.3,371.1,20.4
Jimmy Butler,2023,MIA,SF,13,681.9,604,96.3,188.8,66.4,14.3,...,33.9,78.1,112.0,102.9,29.9,5.1,28.6,27.3,354.1,35.9
Max Strus,2023,MIA,SG,18,553.5,799,58.6,102.9,74.2,70.3,...,14.3,74.2,88.5,43.0,19.5,10.2,29.9,54.7,358.0,18.3
Caleb Martin,2023,MIA,PF,18,514.9,751,66.4,131.5,65.8,29.9,...,48.2,69.0,117.2,50.8,29.9,10.4,26.0,46.9,243.5,16.9
Tyler Herro,2023,MIA,SG,11,444.4,447,63.8,123.7,67.2,37.8,...,10.2,82.0,89.8,37.8,11.9,10.2,31.2,24.7,277.3,27.6
Gabe Vincent,2023,MIA,PG,16,366.5,536,40.4,76.8,68.4,31.2,...,6.8,40.4,45.6,66.4,16.9,3.4,28.6,53.4,201.8,16.3
Dewayne Dedmon,2023,MIA,C,16,265.9,259,43.0,67.7,82.7,10.4,...,18.2,48.2,66.4,15.6,3.4,10.4,11.7,39.1,148.4,29.1
Duncan Robinson,2023,MIA,SF,16,215.5,366,16.9,35.2,62.6,28.6,...,6.8,40.4,45.6,27.3,1.7,0.0,14.3,37.8,131.5,10.8
Nikola Jovic,2023,MIA,SF,9,136.7,189,16.9,32.6,67.7,8.5,...,11.9,23.4,32.6,13.0,8.5,0.0,10.4,14.3,75.5,18.2


In [6]:
create_pdf(df)